In [1]:
import numpy as np
from scipy import optimize
import sympy as sp


def pade_exp(n, s):
    return (1 + 0.5 * n * s) / (1 - 0.5 * n * s)


# Fazer em laplace
def wood_berry(u, s):
    g = sp.Array(
        [
            [
                (12.8 * pade_exp(-1, s)) / (16.7 * s + 1),
                (-18.9 * pade_exp(-3, s)) / (21 * s + 1),
            ],
            [
                (6.6 * pade_exp(-7, s)) / (10.9 * s + 1),
                (-19.4 * pade_exp(-3, s)) / (14.4 * s + 1),
            ],
        ]
    )
    y = g @ u
    return y


def funcao_objetivo(erro):
    f = sum(k * (np.abs(e1) + np.abs(e2)) for k, (e1, e2) in enumerate(erro))
    return f


# Fazer em laplace
def controlador(c, s, erro):
    # Função qualquer aqui só para rodar
    # Depois trocar pelo modelo correto
    u1 = c[0] + c[1] * erro[0] / s + c[2] * erro[0] * s
    u2 = c[3] + c[4] * erro[1] / s + c[5] * erro[1] * s
    return np.array([u1, u2]).T


def inverse_laplace(sis, s, t):
    sis = sp.N(sp.polys.partfrac.apart(sis, s, full=True).doit(), 5)
    for sisf in sis.args:
        v1 = 0
        v2 = 0
        if len(sisf.args):
            v1 = sisf.args[0]
            sisf2 = 1 / sisf.args[1]
            if len(sisf2.args):
                v2 = sisf2.args[0]
        yield v1 * sp.exp(-v2 * t)


# Transformar em função de tempo
def sistema(c, s, t, erro):
    u = controlador(c, s, erro)  # Tenho que mudar aqui
    y = wood_berry(u, s)  # Tenho que mudar aqui
    return [sum(inverse_laplace(y, s, t)) for y in y]


def evaluate_sistem(c, s, t, N):
    erro = np.array([0, 0])
    sis = sistema(c, s, t, erro)
    erros = []
    for k in range(N):
        erro = np.array(
            [
                float(sis[0].evalf(subs={t: k})) - 0.96,
                float(sis[1].evalf(subs={t: k})) - 0.05,
            ]
        )
        sis = sistema(c, s, t, erro)
        erros.append(erro)
    ita = funcao_objetivo(erros)
    return ita


s = sp.symbols("s")
t = sp.symbols("t", positive=True)
# y_target = np.array([0.96, 0.05])
c = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
# erro = y - y_target
N = 500

optimize.minimize(evaluate_sistem, x0=c, args=(s, t, N))
evaluate_sistem(c, s, t, N)